In [1]:
# Add Dependencies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
import requests
import time
from datetime import datetime
import gmaps

# Import the API keys
from config import weather_api_key
from config import g_key



In [2]:
file_to_load = 'weather_data/cities.csv'

city_data_df = pd.read_csv(file_to_load)

city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [3]:
# Configure gmaps to use the API key.

gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the lats and longs

locations = city_data_df[['Lat','Lng']]

# Get the temperature for use as a weighting value

max_temp = city_data_df ['Max Temp']

temps = []

# Assign the figure variable for gmaps - Configure the figure for fit and zoom

fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# Assign the heatmap layer variable to the heatmap_layer attribute

heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating=False,
                                                    max_intensity=300, point_radius=4)

# Add the heatmap layer

fig.add_layer(heat_layer)

# Call the figure to plot the data

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of Humidity
# Get the lats and longs

locations = city_data_df[['Lat','Lng']]

# Get the temperature for use as a weighting value

humidity = city_data_df ['Humidity']

temps = []

# Assign the figure variable for gmaps - Configure the figure for fit and zoom

fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# Assign the heatmap layer variable to the heatmap_layer attribute

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,max_intensity=300, point_radius=4)

# Add the heatmap layer

fig.add_layer(heat_layer)

# Call the figure to plot the data

fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of Cloudiness
# Get the lats and longs

locations = city_data_df[['Lat','Lng']]

# Get the temperature for use as a weighting value

clouds = city_data_df ['Cloudiness']

temps = []

# Assign the figure variable for gmaps - Configure the figure for fit and zoom

fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# Assign the heatmap layer variable to the heatmap_layer attribute

heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False,max_intensity=300, point_radius=4)

# Add the heatmap layer

fig.add_layer(heat_layer)

# Call the figure to plot the data

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of Wind Speed
# Get the lats and longs

locations = city_data_df[['Lat','Lng']]

# Get the temperature for use as a weighting value

wind_speed = city_data_df ['Wind Speed']

temps = []

# Assign the figure variable for gmaps - Configure the figure for fit and zoom

fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# Assign the heatmap layer variable to the heatmap_layer attribute

heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False,max_intensity=300, point_radius=4)

# Add the heatmap layer

fig.add_layer(heat_layer)

# Call the figure to plot the data

fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to specify the temperature range for their trip

min_temp = float(input('What is the minimum temperature you would like for your trip?'))

max_temp = float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?85


In [9]:
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]

preferred_cities_df.notnull().sum()

City_ID       291
City          291
Country       291
Date          291
Lat           291
Lng           291
Max Temp      291
Humidity      291
Cloudiness    291
Wind Speed    291
dtype: int64

In [10]:
# Create a copy of the preferred_cities_df with specific columns

hotel_df = preferred_cities_df[['City','Country','Max Temp','Lat','Lng']].copy()
hotel_df['Hotel Name'] = ''

hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Hasaki,JP,79.29,35.7333,140.8333,
3,Cape Town,ZA,72.91,-33.9258,18.4232,
4,Alofi,NU,75.51,-19.0595,-169.9187,
6,Wewak,PG,79.77,-3.5534,143.6268,
7,Haguenau,FR,75.29,48.8156,7.7905,
...,...,...,...,...,...,...
663,Storm Lake,US,80.15,42.6411,-95.2097,
666,Miles City,US,76.17,46.4083,-105.8406,
667,Japura,ID,75.76,-0.3167,102.3500,
669,Daru,PG,77.88,-9.0763,143.2092,


In [11]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

params = {
    'radius': 5000,
    'type': 'Lodging',
    'key': g_key
}


# Iterate through the DataFrame

for index, row in hotel_df.iterrows():
    # get the location lat & lng
    lat = row['Lat']
    lng = row['Lng']
    
    # Add lat and lng to a location key for our search params
    params['location'] = f'{lat},{lng}'
    
    # Make the request and get JSON response data
    hotels = requests.get(base_url, params=params).json()
    time.sleep(1)
    
    try:
        # Identify the hotels from the results and store the names
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][1]['name']
    except (IndexError):
        print('Hotel not found....  Skipping.')


In [12]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Hasaki,JP,79.29,35.7333,140.8333,Inubosaki Kanko Hotel
3,Cape Town,ZA,72.91,-33.9258,18.4232,Southern Sun Waterfront Cape Town
4,Alofi,NU,75.51,-19.0595,-169.9187,Niue Post
6,Wewak,PG,79.77,-3.5534,143.6268,National Development Bank Wewak Branch
7,Haguenau,FR,75.29,48.8156,7.7905,Kyriad Direct Haguenau
8,Avarua,CK,75.25,-21.2078,-159.7750,Paradise Inn
10,Port Blair,IN,82.18,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
11,Yumen,CN,71.92,40.2833,97.2000,Xiadong No. Liudui
12,Kapaa,US,73.38,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
13,Westport,US,84.02,41.1415,-73.3579,The Westport Inn


In [13]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Inubosaki Kanko Hotel</dd>\n<dt>City</dt><dd>Hasaki</dd>\n<dt>Country</dt><dd>JP</dd>\n<dt>Max Temp</dt><dd>79.29 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Southern Sun Waterfront Cape Town</dd>\n<dt>City</dt><dd>Cape Town</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Max Temp</dt><dd>72.91 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Niue Post</dd>\n<dt>City</dt><dd>Alofi</dd>\n<dt>Country</dt><dd>NU</dd>\n<dt>Max Temp</dt><dd>75.51 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>National Development Bank Wewak Branch</dd>\n<dt>City</dt><dd>Wewak</dd>\n<dt>Country</dt><dd>PG</dd>\n<dt>Max Temp</dt><dd>79.77 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Kyriad Direct Haguenau</dd>\n<dt>City</dt><dd>Haguenau</dd>\n<dt>Country</dt><dd>FR</dd>\n<dt>Max Temp</dt><dd>75.29 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Paradise Inn</dd>\n<dt>City</dt><dd>Avarua</dd>\n<dt>Country</dt><dd>CK</dd>\n<dt>Max Temp</dt><dd>75.25 °F</dd>\n</

In [14]:
# Heatmap of temperature
# Get the lats and longs

locations = hotel_df[['Lat','Lng']]

# Get the temperature for use as a weighting value

max_temp = hotel_df ['Max Temp']

temps = []

# Assign the figure variable for gmaps - Configure the figure for fit and zoom

fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# Assign the heatmap layer variable to the heatmap_layer attribute

heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating=False,
                                                    max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data

fig

Figure(layout=FigureLayout(height='420px'))